In [21]:
import json
import copy
import random
import time

#Functions for generating game states
def generate_next_gameloop(current_state, current_round):
    """
    Generates the next game loop in a game based on the current state.

    :param current_state: A JSON object representing the current state of the game.
    :return: A JSON object representing the next state of the game.
    """
    # Deep copy the current state to avoid modifying the original state
    new_state = copy.deepcopy(current_state)
    # Increment the game loop number
    new_state[f"loop_{current_round + 1}"] = copy.deepcopy(current_state[f"loop_{current_round}"])

    # Process each bike and its agents
    for bike in new_state[f"loop_{current_round + 1}"]['bikes'].values():
        # Check if the Audi intersects this bike
        audi_intersects = new_state[f"loop_{current_round + 1}"]['audi']['target'] == bike['id']
        bike["position"]["x"] = bike["position"]["x"] + random.randint(-10, 10)
        bike["position"]["y"] = bike["position"]["y"] + random.randint(-10, 10)
        # Process each agent on the bike
        for agent in bike['agents'].values():
            # If the Audi intersects, remove all agents from the bike
            if audi_intersects:
                agent['energy'] = 0  # Setting energy to 0 as they are eliminated

            # Update agent's energy and check if they should be moved to bike 0
            if agent['energy'] > 0:
                agent['energy'] -= 1  # Decrease energy by 1 for this loop
                if agent['energy'] == 0:
                    # Move to bike 0
                    agent['bikeID'] = '0x0'
            else:
                # Already on bike 0 or moved to bike 0
                agent['bikeID'] = '0x0'

    # Handle agents on bike 0 (this can be expanded with more logic if needed)
    # Currently, it only ensures they are on bike 0

    return new_state

def generate_initial_state():
    """
    Generates an initial game state with a random number of bikes and agents.

    :return: A JSON object representing the initial state of the game.
    """
    # Define possible colors for agents
    possible_colours = ["Red", "Green", "Blue", "Yellow", "Orange", "Purple", "Pink", "Brown", "Gray", "White"]

    # Initialize the game state
    initial_state = {
        "loop_0": {
            "bikes": {},
            "lootboxes": {},  # This can be populated with lootbox data if needed
            "audi": {
                "target": None,  # No target initially
                "position": {"x": random.randint(-300, 300), "y": random.randint(-300, 300)},
            }
        },
    }

    # Randomly determine the number of bikes
    num_bikes = random.randint(6, 10)

    for bike_id in range(1, num_bikes + 1):
        # Each bike will have a unique ID
        bike_key = f"bike_{bike_id}"

        # Randomly determine the number of agents on this bike
        num_agents = random.randint(3, 9)
        bike_agents = {}

        for agent_id in range(1, num_agents + 1):
            # Each agent will have a unique ID and randomly assigned color
            agent_key = f"agent_{bike_id}_{agent_id}"
            agent_colour = random.choice(possible_colours)

            # Define the agent
            bike_agents[agent_key] = {
                "id": agent_key,
                "groupID": random.randint(0, 8),
                "colour": agent_colour,
                "points": 0,
                "energy": 100,  # Initial energy can be adjusted
                # The rest of the agent's attributes are placeholders
                "bikes": {},
                "forces": {},
                "lootboxChoice": None,
                "lootDistribution": [],
            }

        # Add the bike and its agents to the game state
        initial_state["loop_0"]["bikes"][bike_key] = {
            "id": bike_key,
            "position": {"x": random.randint(-300, 300), "y": random.randint(-300, 300)},
            "agents": bike_agents
        }

    return initial_state


In [22]:
# Generate the initial state and run the game for LOOPS number of loops
initial_state = generate_initial_state()
LOOPS = 10
for i in range(LOOPS):
    initial_state = generate_next_gameloop(initial_state, i)
with open(f"./visualiser/JSON/test2.json", "w") as f:
    json.dump(initial_state, f, indent=4)